# Aula 10 - Análise de Sentimentos e Classificação de Texto

* Identificar diversas aplicações práticas de Análise de Sentimentos em diferentes domínios.
* Compreender o funcionamento básico do algoritmo Naive Bayes para classificação de textos e análise de sentimentos.
* Compreender o funcionamento básico do algoritmo de Árvores de Decisão para classificação de textos.
* Identificar diferentes abordagens para realizar análise de sentimentos.

## Exemplos do roteiro

### Exemplo 1 - Análise de sentimentos com a biblioteca NLTK

In [1]:
# Importanto as bibliotecas

# NLTK -> (Natural Language Toolkit) é uma biblioteca popular de PLN (Processamento de Linguagem Natural) em Python.
# Ela fornece ferramentas para pré-processamento de texto, análise sintática, semântica e léxica, entre outras.
import nltk

# SentimentIntensityAnalyzer -> é uma ferramenta do NLTK baseada no léxico VADER (Valence Aware Dictionary and sEntiment Reasoner).
# Ele é usado para analisar o sentimento de frases em linguagem natural, atribuindo escores de positividade, negatividade e neutralidade.
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Baixando os recursos necessários para executar o analisador de sentimentos

# 'vader_lexicon' -> é um dicionário com palavras e expressões associadas a valores de sentimento.
# É essencial para o funcionamento do SentimentIntensityAnalyzer, pois ele usa esse léxico para atribuir escores às palavras analisadas.
nltk.download('vader_lexicon')

# 'punkt' -> é um recurso utilizado para segmentação de texto em frases e palavras (tokenização).
# Embora não seja usado diretamente no exemplo básico com VADER, é um dos requisitos comuns ao trabalhar com outros recursos de análise textual no NLTK.
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# Inicialização do analisador de sentimentos VADER

# Instância do SentimentIntensityAnalyzer, que utiliza o léxico 'vader_lexicon' como base.
# O analisador calcula a intensidade de sentimentos presentes em frases curtas.
# Para cada texto avaliado, são retornados quatro escores:
# - 'neg': intensidade negativa (0 a 1)
# - 'neu': intensidade neutra (0 a 1)
# - 'pos': intensidade positiva (0 a 1)
# - 'compound': valor composto que resume o sentimento geral da frase (-1 = muito negativo, +1 = muito positivo)
# O método é apropriado para textos curtos e informais, como avaliações e comentários.

sid = SentimentIntensityAnalyzer()

# Conjunto de textos para análise

# Lista com frases que expressam sentimentos variados, simulando comentários reais.
# Os exemplos abrangem polaridades positiva, negativa e neutra, possibilitando observar o comportamento do analisador.

textos = [
    "Este filme é incrível! Eu adorei cada minuto",               # sentimento positivo
    "Odiei este filme. Foi uma perda de tempo.",                  # sentimento negativo
    "O filme foi ok, nada de especial",                           # sentimento neutro
    "Estou muito feliz com o produto, funciona perfeitamente!",   # sentimento positivo
    "Que serviço horrível! Nunca mais compro nessa loja."         # sentimento negativo
]

# As frases serão utilizadas para aplicar o analisador e verificar os escores gerados.


In [3]:
# Aplicação da análise de sentimentos em cada texto da lista

# Percorre todos os textos previamente definidos
for texto in textos:
    # Exibe o conteúdo atual que será analisado
    print(f"Texto: {texto}")

    # Aplica o analisador VADER para calcular os escores de sentimento
    ss = sid.polarity_scores(texto)

    # Mostra os escores gerados (negativo, neutro, positivo e composto)
    print(f"Pontuação: {ss}")

    # Classificação do sentimento com base no valor 'compound':
    # - Valores acima de 0.05 indicam sentimento positivo
    # - Valores abaixo de -0.05 indicam sentimento negativo
    # - Valores entre -0.05 e 0.05 indicam sentimento neutro
    if ss['compound'] >= 0.05:
        print("Sentimento: Positivo")
    elif ss['compound'] <= -0.05:
        print("Sentimento: Negativo")
    else:
        print("Sentimento: Neutro")

    # Separador visual entre as análises
    print("\n")


Texto: Este filme é incrível! Eu adorei cada minuto
Pontuação: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentimento: Neutro


Texto: Odiei este filme. Foi uma perda de tempo.
Pontuação: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentimento: Neutro


Texto: O filme foi ok, nada de especial
Pontuação: {'neg': 0.0, 'neu': 0.694, 'pos': 0.306, 'compound': 0.296}
Sentimento: Positivo


Texto: Estou muito feliz com o produto, funciona perfeitamente!
Pontuação: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentimento: Neutro


Texto: Que serviço horrível! Nunca mais compro nessa loja.
Pontuação: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentimento: Neutro




### Exemplo 2 - Análise de sentimentos de Avaliação de Filmes com Naives Bayes

In [4]:
# Primeira etapa: definição de funções auxiliares para cálculo de frequências e probabilidades

# NumPy é importado por ser uma biblioteca fundamental para operações matemáticas e estatísticas
import numpy as np

# Função para contar a frequência das palavras em avaliações de um determinado sentimento
def contar_palavras(avaliacoes, sentimento):
    # Retorna um dicionário com a frequência de cada palavra que aparece nas avaliações associadas ao sentimento especificado.

    contagem = {}  # Dicionário para armazenar a contagem das palavras

    # Itera sobre as avaliações e seus respectivos rótulos (sentimentos)
    for avaliacao, sent in avaliacoes:
        if sent == sentimento:
            # Converte o texto para minúsculas e divide em palavras
            for palavra in avaliacao.lower().split():
                # Incrementa o contador da palavra, com verificação de existência
                contagem[palavra] = contagem.get(palavra, 0) + 1

    return contagem

# Função para calcular a probabilidade de um sentimento específico
def calcular_probabilidade_sentimento(avaliacoes, sentimento):
    # Retorna a probabilidade de uma avaliação pertencer ao sentimento informado.
    # Calculado como: (número de avaliações com o sentimento) / (total de avaliações)

    total_avaliacoes = len(avaliacoes)  # Número total de avaliações
    contagem_sentimento = sum(1 for _, sent in avaliacoes if sent == sentimento)  # Quantas têm o sentimento desejado
    return contagem_sentimento / total_avaliacoes

# Função para calcular a probabilidade de uma palavra aparecer dado um sentimento (com suavização)
def calcular_probabilidade_palavra_dado_sentimento(palavra, contagem_palavras, total_palavras_sentimento, total_palavras_vocabulario, alpha=1):

    # Aplica a fórmula de probabilidade com suavização de Laplace:
    # P(palavra|sentimento) = (contagem_palavra + alpha) / (total_palavras_sentimento + alpha * total_vocabulario)

    # - contagem_palavras: dicionário com as frequências das palavras do sentimento
    # - total_palavras_sentimento: soma de todas as ocorrências de palavras no sentimento
    # - total_palavras_vocabulario: número total de palavras distintas no vocabulário
    # - alpha: valor de suavização (Laplace), evita probabilidade zero para palavras ausentes

    contagem_palavra = contagem_palavras.get(palavra, 0)  # Retorna a frequência da palavra, ou zero se não houver
    return (contagem_palavra + alpha) / (total_palavras_sentimento + alpha * total_palavras_vocabulario)


In [5]:
# Segunda etapa: função principal para classificar o sentimento de uma nova avaliação

def classificar_avaliacao(avaliacao, avaliacoes):
    # Esta função implementa o algoritmo Naive Bayes para classificar o sentimento de uma nova avaliação textual.
    # Ela usa uma base de dados (avaliacoes) contendo textos rotulados como "Positivo" ou "Negativo".
    # A saída é o sentimento previsto para o novo texto fornecido.

    # Lista de possíveis sentimentos utilizados na base de dados
    sentimentos = ["Positivo", "Negativo"]

    # Dicionário que armazenará a probabilidade final de a nova avaliação pertencer a cada sentimento
    probabilidades_sentimento = {}

    # Calcula a probabilidade a priori P(sentimento) para cada classe
    # Fórmula: quantidade de avaliações com esse sentimento / total de avaliações
    probabilidades_sentimento["Positivo"] = calcular_probabilidade_sentimento(avaliacoes, "Positivo")
    probabilidades_sentimento["Negativo"] = calcular_probabilidade_sentimento(avaliacoes, "Negativo")

    # Conta quantas vezes cada palavra apareceu em avaliações positivas
    contagem_palavras_positivo = contar_palavras(avaliacoes, "Positivo")

    # Conta quantas vezes cada palavra apareceu em avaliações negativas
    contagem_palavras_negativo = contar_palavras(avaliacoes, "Negativo")

    # Soma total de palavras em todas as avaliações positivas
    total_palavras_positivo = sum(contagem_palavras_positivo.values())

    # Soma total de palavras em todas as avaliações negativas
    total_palavras_negativo = sum(contagem_palavras_negativo.values())

    # Cria um vocabulário com todas as palavras distintas da base de avaliações
    # Este vocabulário será usado para aplicar a suavização de Laplace nas probabilidades
    vocabulario = set()
    for avaliacao, _ in avaliacoes:
        for palavra in avaliacao.lower().split():
            vocabulario.add(palavra)
    total_palavras_vocabulario = len(vocabulario)

    # Para cada sentimento, calcular a probabilidade da nova avaliação ser daquele sentimento
    for sentimento in sentimentos:
        # Começa com a probabilidade a priori P(sentimento)
        probabilidade_total = probabilidades_sentimento[sentimento]

        # Seleciona a contagem de palavras e o total de palavras correspondente ao sentimento atual
        contagem_palavras_sentimento = (
            contagem_palavras_positivo if sentimento == "Positivo" else contagem_palavras_negativo
        )
        total_palavras_sentimento = (
            total_palavras_positivo if sentimento == "Positivo" else total_palavras_negativo
        )

        # Para cada palavra presente na nova avaliação...
        for palavra in avaliacao.lower().split():
            # Calcula a probabilidade condicional P(palavra|sentimento) com suavização de Laplace
            probabilidade_palavra = calcular_probabilidade_palavra_dado_sentimento(
                palavra,
                contagem_palavras_sentimento,
                total_palavras_sentimento,
                total_palavras_vocabulario
            )
            # Multiplica a probabilidade atual pela probabilidade condicional da palavra (Naive Bayes)
            probabilidade_total *= probabilidade_palavra

        # Atualiza o valor final da probabilidade para aquele sentimento
        probabilidades_sentimento[sentimento] = probabilidade_total

    # Compara os valores das probabilidades finais e retorna o sentimento com maior valor
    melhor_sentimento = max(probabilidades_sentimento, key=probabilidades_sentimento.get)
    return melhor_sentimento


**Comentário**

Esta função é o coração do modelo Naive Bayes aplicado à análise de sentimentos. Nela, a combinação de **probabilidades a priori** e **probabilidades condicionais suavizadas** (com Laplace) evidencia a aplicação do Teorema de Bayes para problemas reais de PLN.

**Avaliação**

A abordagem, embora simplificada, é didaticamente eficaz, pois mostra como a contagem bruta de palavras pode alimentar um modelo preditivo probabilístico. Essa função também serve como ponte conceitual entre teoria (Bayes) e prática (classificação de texto), sendo um ótimo exemplo para fixar a ideia de que mesmo modelos "ingênuos" podem gerar bons resultados em problemas complexos como interpretação de linguagem natural.

In [6]:
# Terceira etapa – definição do conjunto de dados de treinamento

# Lista de tuplas contendo pares (texto, sentimento)
# Este conjunto simula uma base de dados rotulada manualmente
# Os textos representam avaliações de filmes
# Os sentimentos indicam a classificação atribuída: "Positivo" ou "Negativo"

avaliacoes = [
    ("Filme incrível! Adorei a atuação.", "Positivo"),       # avaliação positiva
    ("Roteiro fraco, não recomendo.", "Negativo"),           # avaliação negativa
    ("Atuação boa, mas filme longo.", "Positivo"),           # avaliação positiva com ressalva
    ("Péssimo! Perdi meu tempo.", "Negativo"),               # avaliação claramente negativa
]


In [9]:
# Quarta etapa – fornecimento de uma nova frase para classificação

# Solicita ao usuário uma nova avaliação textual como entrada
# A frase digitada simula uma opinião real que será analisada
nova_avaliacao = input("O que você achou do filme 101 Dalmatas? ")

# Chama a função de classificação utilizando o modelo treinado anteriormente
# A função retorna o sentimento mais provável associado à nova frase
sentimento = classificar_avaliacao(nova_avaliacao, avaliacoes)

# Exibe o resultado da classificação ao usuário
print(f"A avaliação '{nova_avaliacao}' foi classificada como: {sentimento}")


O que você achou do filme 101 Dalmatas? Maravilhoso
A avaliação 'Maravilhoso' foi classificada como: Positivo


In [8]:
# Quinta etapa – ampliação da base de dados com novas avaliações

# Lista expandida contendo múltiplas avaliações de filmes, rotuladas como "Positivo" ou "Negativo"
# Esse conjunto inclui frases mais longas, complexas e com vocabulário mais variado
# A diversidade de exemplos aumenta a robustez do modelo e melhora a capacidade de generalização

avaliacoes = [
    ("Filme incrível! Adorei a atuação, roteiro e direção.", "Positivo"),
    ("Roteiro fraco, atuação ruim, não recomendo. Perda de tempo.", "Negativo"),
    ("Atuação boa, mas filme um pouco longo e arrastado.", "Positivo"),
    ("Péssimo! Perdi meu tempo, dinheiro e paciência. Horrível.", "Negativo"),
    ("Amei! Elenco maravilhoso, fotografia impecável. Recomendo muito!", "Positivo"),
    ("Chato, previsível e sem graça. Saí no meio do filme.", "Negativo"),
    ("Uma obra-prima! Roteiro inteligente, atuações memoráveis.", "Positivo"),
    ("Cansativo, atuações forçadas, final decepcionante.", "Negativo"),
    ("Divertido e emocionante! Ótimo para assistir com a família.", "Positivo"),
    ("Clichê, sem originalidade, diálogos fracos. Não vale a pena.", "Negativo"),
    ("Surpreendente e envolvente! Me prendeu do início ao fim.", "Positivo"),
    ("Confuso, mal dirigido, edição amadora. Um desastre.", "Negativo"),
    ("Atuação excelente, história cativante, trilha sonora perfeita.", "Positivo"),
    ("Atuações medianas, história arrastada, trilha sonora irritante.", "Negativo"),
    ("Roteiro bem construído, diálogos afiados, direção impecável.", "Positivo"),
    ("Roteiro fraco, diálogos bobos, direção perdida.", "Negativo"),
    ("Uma experiência cinematográfica única! Recomendo a todos os amantes do cinema.", "Positivo"),
    ("Um filme esquecível, sem emoção e sem impacto.", "Negativo"),
    ("Comovente e inspirador! Me fez refletir sobre a vida.", "Positivo"),
    ("Decepcionante e frustrante! Não recomendo nem para passar o tempo.", "Negativo"),
    ("Ótimo, gostei muito", "Positivo")
]


## Desafios do roteiro

Os desafios aqui foram na escrita do script e na execução da análise de sentimentos usando VADER e Naive Bayes:

1. Pré-processamento de Texto Simplificado
O script não aplica técnicas de pré-processamento mais robustas, como remoção de pontuação, stemming ou stopword. Isso afeta a consistência da análise, especialmente em modelos como o Naive Bayes, que são sensíveis a mudanças de vocabulário.

2. Dependência de Amostras Curtas
A análise usando o Sentiment Strength Analyzer (VADER) funciona bem para frases curtas, mas sua precisão pode diminuir ao lidar com textos longos ou ambíguos. Isso limita o script a trabalhar com amostras simples.

3. Banco de Dados Limitado
O pequeno número de exemplos no modelo inicial afeta diretamente a capacidade do classificador Naive Bayes de generalizar para novos dados. Modelos baseados em probabilidade dependem da representação no treinamento.

4. Vocabulário muito específico
Como a classificação estatística depende do vocabulário presente nas frases de treinamento, o roteiro precisa ser cuidadoso para garantir que o vocabulário relevante esteja presente em ambos os conjuntos de dados (conjuntos de treinamento e teste).

5. Interpretação de pontuações compostas
Embora as pontuações compostas sejam úteis para decisões binárias (positivas, neutras ou negativas), sua sensibilidade a mudanças sutis pode ser confusa quando as frases contêm emoções mistas.